In [1]:
# 在Jupyter Notebook中使用backtrader和pyfolio进行高级分析
import backtrader as bt
import pandas as pd
import numpy as np


# 兼容性处理：为旧代码提供 np.NaN 支持
if not hasattr(np, 'NaN'):
    np.NaN = np.nan

import pyfolio as pf

%matplotlib inline


import backtrader as bt
import pandas as pd

class RocStrategy(bt.Strategy):
    params = (
        ('roc_period', 20),   # ROC计算周期
        ('buy_threshold', 0.08),  # 买入阈值
        ('sell_threshold', 0.0),  # 卖出阈值
    )
    def log(self, txt, dt=None, doprint=False):
        if doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))
            
    def stop(self):
        self.log('(roc Period %2d, %.2f,%.2f) Ending Value %.2f' %
                 (self.params.roc_period,self.params.buy_threshold,self.params.sell_threshold, self.broker.getvalue()), doprint=True)

    def __init__(self):
        self.roc = bt.indicators.RateOfChange(self.data.close, period=self.params.roc_period)
        self.order = None

    def next(self):
        # 取消前一个未完成订单
        if self.order:
            self.cancel(self.order)
            self.order = None

        # 没有持仓且ROC突破买入阈值
        if not self.position:
            if self.roc[0] > self.params.buy_threshold:
                # 计算可用资金可买数量
                size = int(self.broker.getcash()*0.98 / self.data.close[0])
                if size > 0:
                    self.order = self.buy(size=size)
                    #print(f'{self.datetime.date()}: BUY {size} shares at {self.data.close[0]:.2f}')
        
        # 有持仓且ROC跌破卖出阈值
        else:
            if self.roc[0] < self.params.sell_threshold:
                self.order = self.sell(size=self.position.size)
                #print(f'{self.datetime.date()}: SELL {self.position.size} shares at {self.data.close[0]:.2f}')



ModuleNotFoundError: No module named 'pyfolio'

In [ ]:
# 加载CSV数据（请替换为你的创业板数据文件路径）
s = '159915'
csv = f'../../data/{s}.csv'


data = pd.read_csv(csv)
data.set_index('date',inplace=True)
data.index = pd.to_datetime(data.index)
data = bt.feeds.PandasData(
        dataname=data,
        fromdate=pd.to_datetime('2010-01-01'),
        todate=pd.to_datetime('2025-12-31'),
        timeframe=bt.TimeFrame.Days
    )

data

In [ ]:
# 设置回测
cerebro = bt.Cerebro()
cerebro.addstrategy(RocStrategy)



cerebro.adddata(data)
cerebro.broker.setcash(100000.0)  # 设置初始资金
cerebro.broker.setcommission(commission=0.001)  # 0.1%佣金

# 添加分析器
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
# 添加PyFolio分析器
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')


print('回测开始...')
results = cerebro.run()
print('回测完成!')

# 获取策略实例
strat = results[0]

# 提取PyFolio分析结果
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

# 打印分析结果
print('\n===== 策略分析 =====')
print(f"夏普比率: {strat.analyzers.sharpe.get_analysis()['sharperatio']:.3f}")
print(f"最大回撤: {strat.analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")
print(f"年化收益率: {strat.analyzers.returns.get_analysis()['rnorm100']:.2f}%")


In [ ]:
(returns+1).cumprod().plot()

In [ ]:
# 定义优化参数范围
# ROC周期: 10-30天，步长为5
# 买入阈值: 0.05-0.10，步长为0.01
# 卖出阈值: -0.02-0.02，步长为0.01
cerebro = bt.Cerebro(optreturn=False)
cerebro.adddata(data)
cerebro.broker.setcash(100000.0)  # 设置初始资金
cerebro.broker.setcommission(commission=0.001)  # 0.1%佣金

cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
strats = cerebro.optstrategy(
    RocStrategy,
    roc_period=range(10, 31, 5),
    buy_threshold=[x/100.0 for x in range(5, 11)],
    sell_threshold=[x/100.0 for x in range(-2, 3)]
)

# 存储所有优化结果
optimization_results = []

# 优化完成回调函数
def on_opt_end(strat):
    print(strat,strat.optimization_results)
    optimization_results.append(strat.optimization_results)
# 设置优化回调
cerebro.optcallback = on_opt_end
        
print("开始参数优化...")
cerebro.run(maxcpus=1)  # 设置maxcpus=1避免多进程问题
print("参数优化完成!")

In [ ]:

print(optimization_results)
df_results = pd.DataFrame(optimization_results)
print(df_results)

# 计算排序分数 (年化收益 * 夏普比率 / 最大回撤)
df_results['score'] = (df_results['annual_return'] * df_results['sharpe_ratio']) / (df_results['max_drawdown'] + 0.01)  # 避免除零

# 按最终净值排序
df_sorted = df_results.sort_values('final_value', ascending=False)

# 输出前10名结果
print("\n优化结果前10名:")
print(df_sorted.head(10)[['roc_period', 'buy_threshold', 'sell_threshold', 'final_value', 'annual_return', 'max_drawdown', 'sharpe_ratio', 'trade_count', 'win_rate', 'score']])

# 找到最佳参数
best_params = df_sorted.iloc[0]
print("\n最佳参数组合:")
print(f"ROC周期: {best_params['roc_period']}天")
print(f"买入阈值: {best_params['buy_threshold']*100:.1f}%")
print(f"卖出阈值: {best_params['sell_threshold']*100:.1f}%")
print(f"预期年化收益: {best_params['annual_return']*100:.2f}%")
print(f"最大回撤: {best_params['max_drawdown']*100:.2f}%")
print(f"夏普比率: {best_params['sharpe_ratio']:.2f}")